In [1]:
##Importing libraries (a.k.a "header files" in hardware based languages such as C++)
#for reading of csv files:
import csv 
#for working with arrays:
import numpy as np
#for plotting graphs 
import matplotlib.pyplot as plt #NOTE: THIS CAN BE DELETED IN THE FUTURE
#for writing into an excel file 
import xlwt
from xlwt import Workbook

#class containting all the functions used in this code

class process():
    
    # function to convert a list containing strings to a list containing floats
    def convert_to_float(list):
            for i in range(0,len(list)):
                if list[i]=="---":
                    list[i]= 'N/A'
                else:    
                    list[i]=float(list[i])
                i+=i
    def convert_to_int(list):
            for i in range(len(list)):
                if list[i]=="---":
                    list[i]= 'N/A'
                else:    
                    list[i]=int(list[i])
                i+=i
                          
    #function to split time data extracted from spreadsheet into month, year and day
    def date_extraction():
            i=0
            for i in range(len(time_data)):
                if time_data[i]=="---":
                    day_of_tip[i]='N/A'
                    month_of_tip[i]='N/A'
                    year_of_tip[i]='N/A'
                    print('N/A in row', i , 'for time extraction')
                else:
                    try:
                        day_of_tip[i]=int((time_data[i][0]+time_data[i][1]))
                        month_of_tip[i]=int((time_data[i][3]+time_data[i][4]))
                        year_of_tip[i]=int(time_data[i][6]+time_data[i][7]+time_data[i][8]+time_data[i][9])
                    except IndexError:
                        pass
                        print('error in row', i)                  

### Defining lists required for the rest of the code
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November', 'December']
days_in_a_month=[31,28,31,30,31,30,31,31,30,31,30,31] 


In [2]:
### Reading of EA 15-min data 

# defining lists where data can be extracted from csv
tip_value_data = [] # value recorded in tipping bucket as per 15-min EA records
time_data = [] # time of tip as per 15-min EA records

#reading CSV file
with open('Levens-BridgeEndtelemetry.RE[RainfallEvent].15min.csv', 'r') as file:
    reader = csv.reader(file)
    row_count=0   
    for lines in reader:
        if(row_count > 20): 
            time_data.append(lines[0]) # this data is stored as a string currently
            tip_value_data.append(lines[1]) # this data is stored as a string currently
        row_count+=1
        
#converting tip data from string type (as mentioned above) to float
process.convert_to_float(tip_value_data)

year_of_tip=np.zeros(len(time_data))
month_of_tip=np.zeros(len(time_data))
day_of_tip=np.zeros(len(time_data))

process.date_extraction() #execution of extraction from time_data and filling the three 0-lists above

###Counting the number of storms

storm_count_total=0
storm_count_month=np.zeros(12)
min_yr=int(min(year_of_tip))
max_yr=int(max(year_of_tip))
storm_count_year=np.zeros(max_yr-min_yr+1)
missing_data_count_year=np.zeros(len(storm_count_year))
process.convert_to_int(missing_data_count_year)

#counting the duration and depth of each storm:
storm_depth_duration_list=[[],[],[],[],[]] #format (depth:duration:dd:mm:yy)
index_tracker=[] # counting the first instances of each storm to reduce the number of loops

for i in range (0, len(tip_value_data)):
    if tip_value_data[i]=='N/A': # Counting number of rows of missing data per year 
        temp_val=abs(int(year_of_tip[i]-min_yr))
        missing_data_count_year[temp_val]=missing_data_count_year[temp_val]+1
        del temp_val
    #elif tip_value_data[i]!=0 and tip_value_data[i-1]==0 and tip_value_data[i-2]==0 and tip_value_data[i-3]==0 and tip_value_data[i-4]==0:
    elif (tip_value_data[i]!=0 and
          tip_value_data[i-1]==0 and
          tip_value_data[i-2]==0):
        #counting number of storms:
        storm_count_total+=1
        temp_val=int(month_of_tip[i]-1) #since lists start from 0 in python
        storm_count_month[temp_val]=storm_count_month[temp_val]+1
        del temp_val        
        temp_val=abs(int(year_of_tip[i]-min_yr))
        storm_count_year[temp_val]= storm_count_year[temp_val]+1
        #counting day, month, year for ddf:
        storm_depth_duration_list[2].append(day_of_tip[i])
        storm_depth_duration_list[3].append(month_of_tip[i])
        storm_depth_duration_list[4].append(year_of_tip[i])
        index_tracker.append(i)  #to reduce the number of iterations      
process.convert_to_int(storm_depth_duration_list[3])

##Extracting Duration:
gaps=[]
depth_vals=[] 
for vals in range(0,len(index_tracker)):
    count_duration=1    
    i=index_tracker[vals]
    count_depth=tip_value_data[i]
    while i<(index_tracker[vals]+1000) and i<(len(tip_value_data)-1): #assuming that no storm can last more than 1000 rows (or 10 days)
        if tip_value_data[i+1]!=0 or tip_value_data[i+2]!=0:
            count_duration+=1
            if (tip_value_data[i+1])!="N/A":
                count_depth=(count_depth+tip_value_data[i+1]) 
        else:
            break
        i+=1    
    gaps.append(count_duration)
    depth_vals.append(count_depth)
    del count_duration
    del count_depth
storm_depth_duration_list[1]=[element*15 for element in gaps]
storm_depth_duration_list[0]=depth_vals


In [3]:
# Monthly analysis of rainfall - EA Records

avg_storms_per_month=(storm_count_month/len(storm_count_year))

avg_storms_per_day=[(avg_storms_per_month[i]/days_in_a_month[i]) for i in range(len(avg_storms_per_month))] #note: this needs to be changed to account for leap years

##### depth and duration per month:
total_depth_per_month=np.zeros(12)
total_duration_per_month=np.zeros(12)
for i in range(0,len(storm_depth_duration_list[4])):
    temp_index=storm_depth_duration_list[3][i]-1
    total_depth_per_month[temp_index]=total_depth_per_month[temp_index]+storm_depth_duration_list[0][i]
    total_duration_per_month[temp_index]=total_duration_per_month[temp_index]+storm_depth_duration_list[1][i]

avg_depth_per_month=[total_depth_per_month[i]/len(storm_count_year) for i in range(0,len(total_depth_per_month))]
avg_duration_per_month=[total_duration_per_month[i]/(len(storm_count_year)*60) for i in range(0,len(total_duration_per_month))]


In [4]:
### Annual trends in rainfall - EA Records
import numpy as np
x=np.zeros(len(storm_count_year))
for i in range(0,len(storm_count_year)): #to plot values in X-Axis of graph
    x[i]=min_yr+i

incomplete_years_list= [x[0],x[-1]]

# Total Number of storms per month
colour_index=["grey" for i in range(len(x))]
for val in range(len(missing_data_count_year)):
    if missing_data_count_year[val]>1500: #i.e. at least one month's data missing from dataset
        incomplete_years_list.append(x[val])
        colour_index[val]="red"
incomplete_years_list.sort()

for val in range(len(x)):
    if x[val] in incomplete_years_list:
        colour_index[val]="red"

#Total Rainfall per year 
total_depth_per_year=[]
total_duration_per_year=[]
for i in range(0,len(storm_depth_duration_list[4])):
    if storm_depth_duration_list[4][i]!=storm_depth_duration_list[4][i-1]:
        total_depth_per_year.append(storm_depth_duration_list[0][i])
        total_duration_per_year.append(storm_depth_duration_list[1][i])
    else:
        total_depth_per_year[-1]=total_depth_per_year[-1]+storm_depth_duration_list[0][i]
        total_duration_per_year[-1]=total_duration_per_year[-1]+storm_depth_duration_list[1][i]

total_duration_per_year=[i/60 for i in total_duration_per_year]



In [9]:
###Daily records - Met Office

daily_records=[[],[],[],[],[]] #format is dd:mm:yy:value
#Note that for the above array is deleted later in the code.

#reading CSV file
with open('Underbarrow,OldVicarage.RF_MO[RainfallMetOffice].1days.csv', 'r') as file:
    reader = csv.reader(file)
    row_count=0   
    for lines in reader:
        if(row_count > 20): 
            daily_records[0].append(lines[0]) # this data is stored as a string currently
            daily_records[4].append(lines[1]) # this data is stored as a string currently
            #extracting date
            date=daily_records[0][-1][0]+daily_records[0][-1][1]
            daily_records[1].append(date)
            #extracting month
            mon=daily_records[0][-1][3]+daily_records[0][-1][4]
            daily_records[2].append(mon)
            #extracting year 
            year=daily_records[0][-1][6]+daily_records[0][-1][7]+daily_records[0][-1][8]+daily_records[0][-1][9]
            daily_records[3].append(year)           
            del date, mon, year
        row_count+=1

#Removing all records where the gauge was broken:
final=len(daily_records[3])
daily_export=[[],[],[],[]]
for num in range(0,final):
    if daily_records[4][num]!="---":
        daily_export[0].append(daily_records[1][num])
        daily_export[1].append(daily_records[2][num])
        daily_export[2].append(daily_records[3][num])
        daily_export[3].append(daily_records[4][num])
del daily_records

process.convert_to_float(daily_export[3]) #converting the tip data into floats

#defining min and max years from the list 
min_yr=int(daily_export[2][0])
max_yr=int(daily_export[2][-1])
yr_range=max_yr-min_yr+1
year_list=list(range(min_yr,max_yr+1))

#calculating total rainfall per year 
annual_rainfall=np.zeros(yr_range)# array containing annual total rainfall (mm)
#monthly_rainfall_per_year = [np.zeros(12)]*yr_range # order: Winter, Spring, Summer, Autumn 
monthly_rainfall_per_year = []
[monthly_rainfall_per_year.append(np.zeros(12)) for vals in range(yr_range)] # the last index is reserved for counting of data
difference=0
check=np.zeros(yr_range)
for vals in range (0,len(daily_export[2])):
    difference = int(daily_export[2][vals]) - min_yr
    annual_rainfall[difference]=annual_rainfall[difference]+daily_export[3][vals]
    month=int(daily_export[1][vals])-1
    monthly_rainfall_per_year[difference][month]=monthly_rainfall_per_year[difference][month]+daily_export[3][vals]
    check[difference]+=1
    del difference, month

#deleting incomplete years from annual and monthly series:
daily_tip_export=[[],[]] #year, value
monthly_tip_export=[]
for vals in range(len(check)):
    if check[vals]>360:
        daily_tip_export[0].append(min_yr+vals)
        daily_tip_export[1].append(annual_rainfall[vals])
        monthly_tip_export.append(monthly_rainfall_per_year[vals])
#print(len(daily_tip_export[0])) 


In [10]:
###How many wet days? 
#annual wet day series:
annual_wet_days=np.zeros(len(daily_tip_export[0])) 
#monthly wet day series:
monthly_wet_days=[]
for i in range(len(daily_tip_export[0])):
    monthly_wet_days.append(np.zeros(12))
#counting wet days and years
for yrs in range(0,len(daily_export[2])):
    if int(daily_export[2][yrs]) in daily_tip_export[0]:
        if daily_export[3][yrs]!=0:
            annual_wet_days[daily_tip_export[0].index(int(daily_export[2][yrs]))]+=1
            monthly_wet_days[daily_tip_export[0].index(int(daily_export[2][yrs]))][int(daily_export[1][yrs])-1]+=1

In [11]:
#Exporting Data into Workbook

# Workbook is created
wb = Workbook()
  
# add_sheet is used to create sheet.

sheet1=wb.add_sheet("ddf")
sheet1.write(0, 0, 'Year')
sheet1.write(0, 1, 'Number of Storm')
sheet1.write(0, 2, 'Total Depth (mm)')
sheet1.write(0,3,"Total Duration(hrs)")


storm_count_year_export=list(storm_count_year)
ddf_export=[[],[],[],[]]
for num in range(0,len(total_depth_per_year)):    
    if x[num] not in incomplete_years_list:
        ddf_export[0].append(x[num])
        ddf_export[1].append(storm_count_year[num])
        ddf_export[2].append(total_depth_per_year[num])
        ddf_export[3].append(total_duration_per_year[num])

"""#removing incomplete years from the list
for num in range(0,len(total_depth_per_year)):    
    if x[num] in incomplete_years_list:
        storm_count_year_export.remove(storm_count_year[num])
        total_depth_per_year.remove(total_depth_per_year[num])
        total_duration_per_year.remove(total_duration_per_year[num])"""

#removing incomplete years from the list        
for num in range(0,len(total_depth_per_year)):
    if x[num] not in incomplete_years_list:
        sheet1.write(num+1,0,x[num])
        sheet1.write(num+1,1,storm_count_year[num])
        sheet1.write(num+1,2,total_depth_per_year[num])
        sheet1.write(num+1,3,total_duration_per_year[num])

sheet2 = wb.add_sheet("total rainfall by month")

sheet2.write(0,0,"Month")
sheet2.write(0,1,"Avg depth per Month (mm)")
sheet2.write(0,2,"Avg duration per Month (hrs)")
sheet2.write(0,3,"Number of storms per Month")

for i in range (len(months)):
    sheet2.write(i+1,0,months[i])
    sheet2.write(i+1,1,avg_depth_per_month[i])
    sheet2.write(i+1,2,avg_duration_per_month[i])
    sheet2.write(i+1,3,avg_storms_per_month[i])

sheet3 = wb.add_sheet("seasonal")

sheet3.write(0,0,"Year")
sheet3.write(0,1,"Total Rainfall (mm)")
sheet3.write(0,2,"Rainfall in January (mm)")
sheet3.write(0,3,"Rainfall in Feb (mm)")
sheet3.write(0,4,"Rainfall in March (mm)")
sheet3.write(0,5,"Rainfall in April (mm)")
sheet3.write(0,6,"Rainfall in May (mm)")
sheet3.write(0,7,"Rainfall in June (mm)")
sheet3.write(0,8,"Rainfall in July (mm)")
sheet3.write(0,9,"Rainfall in August (mm)")
sheet3.write(0,10,"Rainfall in Sept (mm)")
sheet3.write(0,11,"Rainfall in Oct (mm)")
sheet3.write(0,12,"Rainfall in Nov (mm)")
sheet3.write(0,13,"Rainfall in Dec (mm)")

for i in range(len(daily_tip_export[0])):
    sheet3.write(i+1,0,daily_tip_export[0][i])
    sheet3.write(i+1,1,daily_tip_export[1][i])
    sheet3.write(i+1,2,monthly_tip_export[i][0])
    sheet3.write(i+1,3,monthly_tip_export[i][1])
    sheet3.write(i+1,4,monthly_tip_export[i][2])
    sheet3.write(i+1,5,monthly_tip_export[i][3])
    sheet3.write(i+1,6,monthly_tip_export[i][4])
    sheet3.write(i+1,7,monthly_tip_export[i][5])
    sheet3.write(i+1,8,monthly_tip_export[i][6])
    sheet3.write(i+1,9,monthly_tip_export[i][7])
    sheet3.write(i+1,10,monthly_tip_export[i][8])
    sheet3.write(i+1,11,monthly_tip_export[i][9])
    sheet3.write(i+1,12,monthly_tip_export[i][10])
    sheet3.write(i+1,13,monthly_tip_export[i][11])

#number of wet days 
sheet4 = wb.add_sheet("Wet days")

sheet4.write(0,0,"Year")
sheet4.write(0,1,"Total wet days")
sheet4.write(0,2,"Jan")
sheet4.write(0,3,"Feb")
sheet4.write(0,4,"March")
sheet4.write(0,5,"April")
sheet4.write(0,6,"May")
sheet4.write(0,7,"June")
sheet4.write(0,8,"July")
sheet4.write(0,9,"August")
sheet4.write(0,10,"Sept")
sheet4.write(0,11,"Oct")
sheet4.write(0,12,"Nov")
sheet4.write(0,13,"Dec")

for i in range(len(annual_wet_days)):
    sheet4.write(i+1,0,daily_tip_export[0][i])
    sheet4.write(i+1,1,annual_wet_days[i])
    sheet4.write(i+1,2,monthly_wet_days[i][0])
    sheet4.write(i+1,3,monthly_wet_days[i][1])
    sheet4.write(i+1,4,monthly_wet_days[i][2])
    sheet4.write(i+1,5,monthly_wet_days[i][3])
    sheet4.write(i+1,6,monthly_wet_days[i][4])
    sheet4.write(i+1,7,monthly_wet_days[i][5])
    sheet4.write(i+1,8,monthly_wet_days[i][6])
    sheet4.write(i+1,9,monthly_wet_days[i][7])
    sheet4.write(i+1,10,monthly_wet_days[i][8])
    sheet4.write(i+1,11,monthly_wet_days[i][9])
    sheet4.write(i+1,12,monthly_wet_days[i][10])
    sheet4.write(i+1,13,monthly_wet_days[i][11])

wb.save('Pair5.xls')

